# Example: Standard Quantum Measurement Process Tomography on 1-Qubit with Cvxpy 

In [1]:
import numpy as np

from quara.objects.qoperation_typical import (
    generate_qoperation,
    generate_qoperation_object,
)
from quara.objects.composite_system_typical import generate_composite_system
from quara.protocol.qtomography.standard.standard_qmpt import (
    StandardQmpt,
)

In [2]:
# System
num_qubits = 1
c_sys = generate_composite_system(mode="qubit", num=num_qubits)

In [3]:
# Tester Objects
state_names = ["x0", "y0", "z0", "z1"]
povm_names = ["x", "y", "z"]

tester_states = [
    generate_qoperation_object(
        mode="state", object_name="state", name=name, c_sys=c_sys
    )
        for name in state_names
]
tester_povms = [
    generate_qoperation_object(
        mode="povm", object_name="povm", name=name, c_sys=c_sys
    )
    for name in povm_names
]

In [4]:
# True Object
true_object_name = "z-type2"
true_object = generate_qoperation(
    mode="mprocess", name=true_object_name, c_sys=c_sys
)

16it [00:00, 516.13it/s]


In [5]:
# Qmpt
on_para_eq_constraint = True
qmpt = StandardQmpt(
    states=tester_states,
    povms=tester_povms,
    num_outcomes=true_object.num_outcomes,
    on_para_eq_constraint=on_para_eq_constraint,
    schedules="all",
)

In [6]:
# empi_dists
prob_dists = qmpt.calc_prob_dists(true_object)
empi_dists = [(10, prob_dist) for prob_dist in prob_dists]

## Qmpt with Cvxpy

In [7]:
from quara.interface.cvxpy.qtomography.standard.loss_function import (
    CvxpyLossFunctionOption,
    CvxpyRelativeEntropy,
    CvxpyUniformSquaredError,
    CvxpyApproximateRelativeEntropyWithZeroProbabilityTerm,
)
from quara.interface.cvxpy.qtomography.standard.minimization_algorithm import (
    CvxpyMinimizationAlgorithm,
    CvxpyMinimizationAlgorithmOption,
)
from quara.interface.cvxpy.qtomography.standard.estimator import (
    CvxpyLossMinimizationEstimator,
)

In [8]:
# Loss and Algorithm
mode_constraint = "physical"
#mode_constraint = "unconstraint"
name_solver = "mosek"
#name_solver = "scs"
#name_solver = "cvxopt"

loss = CvxpyRelativeEntropy()
#loss = CvxpyUniformSquaredError()
#loss = CvxpyApproximateRelativeEntropyWithZeroProbabilityTerm()
loss_option = CvxpyLossFunctionOption()
algo = CvxpyMinimizationAlgorithm()
algo_option = CvxpyMinimizationAlgorithmOption(
    name_solver=name_solver, mode_constraint=mode_constraint
)
estimator = CvxpyLossMinimizationEstimator()

In [9]:
import numpy.testing as npt

In [10]:
# Estimation
sqt = qmpt
result = estimator.calc_estimate(
    qtomography=sqt,
    empi_dists=empi_dists,
    loss=loss,
    loss_option=loss_option,
    algo=algo,
    algo_option=algo_option,
    is_computation_time_required=True,
)
var_estimate = result.estimated_var

# Test
actual = var_estimate
expected = true_object.to_var()

decimal = 1e-8
npt.assert_almost_equal(actual, expected, decimal=decimal)
print("\n")
print("mode_constraint =", mode_constraint)
print("actual          =", actual)
print("expected        =", expected)
print("squared_error   =", np.inner(actual-expected, actual-expected))



mode_constraint = physical
actual          = [ 5.00000000e-01 -1.17075758e-08 -2.73127269e-08  4.99999998e-01
  1.24661182e-09 -5.37708953e-08 -6.52579303e-10  1.24659636e-09
 -1.06140667e-09  6.52578396e-10 -5.37708933e-08 -1.06143495e-09
  4.99999997e-01 -1.12812768e-08 -2.68864282e-08  4.99999997e-01
 -4.32131960e-08 -8.19872995e-09  2.05428181e-08  4.32131999e-08
 -5.65723061e-08  2.05428257e-08  8.19872646e-09  5.65723076e-08
  4.99999997e-01  1.21338884e-08  2.77390462e-08 -4.99999997e-01]
expected        = [ 0.5  0.   0.   0.5  0.   0.   0.   0.   0.   0.   0.   0.   0.5  0.
  0.   0.5  0.   0.   0.   0.   0.   0.   0.   0.   0.5  0.   0.  -0.5]
squared_error   = 1.9590566247416924e-14
